In [1]:
from itertools import permutations
import numpy as np
from time import time
from dlx import DLX
from random import shuffle

# Project Euler problem 179 - Find the largest 0 to 9 pandigital that can be formed by concatenating products

[Link to problem on Project Euler homepage](https://projecteuler.net/problem=170)

## Description

Take the number 6 and multiply it by each of 1273 and 9854:

$$
6 \times 1273 = 7638 \\
6 \times 9854 = 59124
$$

By concatenating these products we get the 1 to 9 pandigital 763859124. We will call 763859124 the "concatenated product of $6$ and $(1273,9854)$". Notice too, that the concatenation of the input numbers, $612739854$, is also $1$ to $9$ pandigital.

The same can be done for 0 to 9 pandigital numbers.

What is the largest 0 to 9 pandigital 10-digit concatenated product of an integer with two or more other integers, such that the concatenation of the input numbers is also a 0 to 9 pandigital 10-digit number?

In [43]:
digits: set[int] = set([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

def numberToDigits(input: int) -> list[int]:
    return [int(digit) for digit in str(input)]

def digitsToNumber(input: list[int]) -> int:
    input.reverse()
    return sum([pow(10, i)*digit for i, digit in enumerate(input)])

def digitsToIndices(input1: set[int], input2: set[int]) -> list[int]:
    return list(input1) + [i+10 for i in input2]

def generateInputDigits(commonNumber: int):
    commonDigits: set[int] = set(numberToDigits(commonNumber))
    inputSet: set[int] = digits - commonDigits
    
    output = [(commonNumber, None, digitsToIndices(commonDigits, set()))]
    for r in range(1, len(inputSet)+1):
        for inputDigits in permutations(inputSet, r):
            if inputDigits[0] == 0:
                continue # number cannot begin with 0
            inputNumber = digitsToNumber(list(inputDigits))
            outputNumber = inputNumber*commonNumber
            if outputNumber > 10**10-1:
                continue ## outputnumber cannot be larger than 9,999,999,999
            outputDigits = numberToDigits(outputNumber)
            if len(outputDigits) != len(set(outputDigits)):
                continue # no repeated digits allowed in output number
            output.append((inputNumber, outputNumber, digitsToIndices(inputDigits, outputDigits)))
    return output

def findSolution(input: int):
    seconds = time()
    inputDigits = numberToDigits(input)
    if len(inputDigits) != len(set(inputDigits)):
        return None
    if input % 3 != 0:
        return None
    inputDigits = generateInputDigits(input)
    
    columns = list(range(20))
    incidenceMatrix = [i[2] for i in inputDigits]
    #shuffle(incidenceMatrix)
    cover = DLX(columns, incidenceMatrix)
    solutions = cover.run_search(sort=False)
    outputSolutions = []
    for solution in solutions:
        if len(solution) > 2:
            outputSolutions.append([(inputDigits[i][0], inputDigits[i][1]) for i in solution[1:]])
    
    seconds = time() - seconds

    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)

    print("")
    print("input number {}".format(input))
    print("number of solutions {}".format(len(outputSolutions)))
    print("cover Matrix size", len(incidenceMatrix))
    print("Total number of calls to recursive search: {}".format(sum(cover.kcount)))
    print("Time elapsed = {:.0f}h {:.0f}m {:f}s".format(hours, minutes, seconds))
    
    return outputSolutions

def findConcatenatedProduct(solution):
    return int(''.join(sorted([str(out) for (inp, out) in solution], reverse=True)))

def findMaxConcatenatedProduct(solutions):
    mx = 0
    for solution in solutions:
        temp = findConcatenatedProduct(solution)
        if temp > mx:
            mx = temp
    return mx

In [47]:
maxInput = 0
maxConcProduct = 0
for i in range(10, 1000):
    solution = findSolution(i)
    if solution is not None:
        
        newConcProd = findMaxConcatenatedProduct(solution)
        if newConcProd > maxConcProduct:
            maxInput = i
            maxConcProduct = newConcProd
        print("   this max {} {}".format(i, newConcProd))
        print("updated max {} {}".format(maxInput, maxConcProduct))


input number 12
number of solutions 220
cover Matrix size 2663
Total number of calls to recursive search: 1817
Time elapsed = 0h 0m 11.859360s
updated max 12 9716832540
updated max 12 9716832540

input number 15
number of solutions 54
cover Matrix size 2956
Total number of calls to recursive search: 1310
Time elapsed = 0h 0m 15.056961s
updated max 15 9724618530
updated max 15 9724618530

input number 18
number of solutions 99
cover Matrix size 2329
Total number of calls to recursive search: 1433
Time elapsed = 0h 0m 10.446808s
updated max 18 9847150236
updated max 18 9847150236

input number 21
number of solutions 4
cover Matrix size 2010
Total number of calls to recursive search: 1336
Time elapsed = 0h 0m 6.574517s
updated max 21 9470512863
updated max 18 9847150236

input number 24
number of solutions 25
cover Matrix size 2009
Total number of calls to recursive search: 1306
Time elapsed = 0h 0m 6.956334s
updated max 24 9517684320
updated max 18 9847150236

input number 27
number of 